In [267]:
import pandas as pd
pd.set_option('display.max_columns', 30)

In [268]:
weekly = pd.read_parquet('data/weekly.parquet')

In [269]:
weekly.columns

Index(['player_id', 'player_name', 'player_display_name', 'position',
       'position_group', 'headshot_url', 'recent_team', 'season', 'week',
       'season_type', 'opponent_team', 'completions', 'attempts',
       'passing_yards', 'passing_tds', 'interceptions', 'sacks', 'sack_yards',
       'sack_fumbles', 'sack_fumbles_lost', 'passing_air_yards',
       'passing_yards_after_catch', 'passing_first_downs', 'passing_epa',
       'passing_2pt_conversions', 'pacr', 'dakota', 'carries', 'rushing_yards',
       'rushing_tds', 'rushing_fumbles', 'rushing_fumbles_lost',
       'rushing_first_downs', 'rushing_epa', 'rushing_2pt_conversions',
       'receptions', 'targets', 'receiving_yards', 'receiving_tds',
       'receiving_fumbles', 'receiving_fumbles_lost', 'receiving_air_yards',
       'receiving_yards_after_catch', 'receiving_first_downs', 'receiving_epa',
       'receiving_2pt_conversions', 'racr', 'target_share', 'air_yards_share',
       'wopr', 'special_teams_tds', 'fantasy_points

In [270]:
context_cols = ['player_id', 'player_display_name', 'position',
       'position_group',
]
game_context = ['week','season_type', 'season', 'opponent_team', 'recent_team', ]
stat_cols = ['completions', 'attempts',
       'passing_yards', 'passing_tds', 'interceptions', 'sacks', 'sack_yards',
       'sack_fumbles', 'sack_fumbles_lost', 'passing_air_yards',
       'passing_yards_after_catch', 'passing_first_downs', 'passing_epa',
       'passing_2pt_conversions', 'pacr', 'dakota', 'carries', 'rushing_yards',
       'rushing_tds', 'rushing_fumbles', 'rushing_fumbles_lost',
       'rushing_first_downs', 'rushing_epa', 'rushing_2pt_conversions',
       'receptions', 'targets', 'receiving_yards', 'receiving_tds',
       'receiving_fumbles', 'receiving_fumbles_lost', 'receiving_air_yards',
       'receiving_yards_after_catch', 'receiving_first_downs', 'receiving_epa',
       'receiving_2pt_conversions', 'racr', 'target_share', 'air_yards_share',
       'wopr', 'special_teams_tds', 'fantasy_points', 'fantasy_points_ppr']
rush_stat_cols = [
     'carries', 'rushing_yards',
       'rushing_tds', 'rushing_fumbles', 'rushing_fumbles_lost',
       'rushing_first_downs', 'rushing_epa', 'rushing_2pt_conversions',
       'receptions', 'targets', 'receiving_yards', 'receiving_tds',
       'fantasy_points', 'fantasy_points_ppr'
]

In [271]:
bark = weekly.loc[weekly.player_display_name=='Saquon Barkley']

In [272]:
weekly.groupby(context_cols)[['fantasy_points','fantasy_points_ppr']].rolling(5, min_periods=1).mean().reset_index()

,player_id,player_display_name,position,position_group,level_4,fantasy_points,fantasy_points_ppr
0,00-0000108,David Akers,K,SPEC,0,4.560000,4.560000
1,00-0000865,Charlie Batch,QB,QB,0,1.000000,1.000000
2,00-0000865,Charlie Batch,QB,QB,1,9.520000,9.520000
3,00-0000865,Charlie Batch,QB,QB,2,7.693334,7.693334
4,00-0000865,Charlie Batch,QB,QB,1,5.720000,5.720000
...,...,...,...,...,...,...,...
78231,00-0039921,Trey Benson,RB,RB,2978,2.325000,2.825000
78232,00-0039921,Trey Benson,RB,RB,2979,2.380000,2.780000
78233,00-0039921,Trey Benson,RB,RB,2980,2.220000,2.420000
78234,00-0039921,Trey Benson,RB,RB,2981,4.180000,4.380000


In [273]:
bark[context_cols + game_context + rush_stat_cols]

,player_id,player_display_name,position,position_group,week,season_type,season,opponent_team,recent_team,carries,rushing_yards,rushing_tds,rushing_fumbles,rushing_fumbles_lost,rushing_first_downs,rushing_epa,rushing_2pt_conversions,receptions,targets,receiving_yards,receiving_tds,fantasy_points,fantasy_points_ppr
5196,00-0034844,Saquon Barkley,RB,RB,1,REG,2018,JAX,NYG,18,106.0,1,0.0,0.0,3.0,0.542650,0,2,6,22.0,0,18.799999,20.799999
5197,00-0034844,Saquon Barkley,RB,RB,2,REG,2018,DAL,NYG,11,28.0,0,0.0,0.0,4.0,-1.857629,0,14,16,80.0,0,10.800000,24.799999
5198,00-0034844,Saquon Barkley,RB,RB,3,REG,2018,HOU,NYG,17,82.0,1,0.0,0.0,4.0,-1.259937,0,5,5,35.0,0,17.700001,22.700001
5199,00-0034844,Saquon Barkley,RB,RB,4,REG,2018,NO,NYG,10,44.0,1,0.0,0.0,2.0,0.069161,0,6,8,56.0,0,16.000000,22.000000
5200,00-0034844,Saquon Barkley,RB,RB,5,REG,2018,CAR,NYG,15,48.0,0,0.0,0.0,2.0,-6.009614,0,4,4,81.0,2,24.900000,28.900000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
819,00-0034844,Saquon Barkley,RB,RB,7,REG,2024,NYG,PHI,17,176.0,1,0.0,0.0,6.0,8.977637,0,2,3,11.0,0,24.700001,26.700001
820,00-0034844,Saquon Barkley,RB,RB,8,REG,2024,CIN,PHI,22,108.0,0,0.0,0.0,5.0,0.820148,0,1,1,3.0,0,11.100000,12.100000
821,00-0034844,Saquon Barkley,RB,RB,9,REG,2024,JAX,PHI,27,159.0,1,1.0,1.0,5.0,-4.333561,0,3,4,40.0,1,29.900000,32.900002
822,00-0034844,Saquon Barkley,RB,RB,10,REG,2024,DAL,PHI,14,66.0,0,0.0,0.0,4.0,0.941058,0,1,1,12.0,0,7.800000,8.800000


In [274]:
player_fantasy = weekly[context_cols + game_context + rush_stat_cols].copy(deep=True)
player_fantasy

,player_id,player_display_name,position,position_group,week,season_type,season,opponent_team,recent_team,carries,rushing_yards,rushing_tds,rushing_fumbles,rushing_fumbles_lost,rushing_first_downs,rushing_epa,rushing_2pt_conversions,receptions,targets,receiving_yards,receiving_tds,fantasy_points,fantasy_points_ppr
0,00-0000865,Charlie Batch,QB,QB,2,REG,2010,TEN,PIT,1,0.0,0,1.0,0.0,0.0,-0.982634,0,0,0,0.0,0,1.000000,1.000000
1,00-0000865,Charlie Batch,QB,QB,3,REG,2010,TB,PIT,5,26.0,0,0.0,0.0,2.0,3.388114,0,0,0,0.0,0,18.040001,18.040001
2,00-0000865,Charlie Batch,QB,QB,4,REG,2010,BAL,PIT,1,4.0,0,0.0,0.0,0.0,-0.714745,0,0,0,0.0,0,4.040000,4.040000
3,00-0001231,Josh Bidwell,P,SPEC,1,REG,2010,DAL,WAS,1,0.0,0,1.0,0.0,0.0,-3.118365,0,0,0,0.0,0,0.000000,0.000000
4,00-0001545,Todd Bouman,QB,QB,7,REG,2010,KC,JAX,2,8.0,0,0.0,0.0,1.0,2.377490,0,0,0,0.0,0,13.680000,13.680000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2978,00-0039921,Trey Benson,RB,RB,4,REG,2024,WAS,ARI,9,50.0,0,1.0,0.0,3.0,-1.033874,0,0,0,0.0,0,5.000000,5.000000
2979,00-0039921,Trey Benson,RB,RB,6,REG,2024,GB,ARI,5,26.0,0,0.0,0.0,1.0,-0.513847,0,0,0,0.0,0,2.600000,2.600000
2980,00-0039921,Trey Benson,RB,RB,8,REG,2024,MIA,ARI,1,10.0,0,0.0,0.0,1.0,0.690985,0,0,0,0.0,0,1.000000,1.000000
2981,00-0039921,Trey Benson,RB,RB,9,REG,2024,CHI,ARI,8,37.0,1,0.0,0.0,2.0,0.168990,0,1,1,18.0,0,11.500000,12.500000


In [275]:
test = player_fantasy.groupby(context_cols)[rush_stat_cols].rolling(5).mean()

In [276]:
player_fantasy

,player_id,player_display_name,position,position_group,week,season_type,season,opponent_team,recent_team,carries,rushing_yards,rushing_tds,rushing_fumbles,rushing_fumbles_lost,rushing_first_downs,rushing_epa,rushing_2pt_conversions,receptions,targets,receiving_yards,receiving_tds,fantasy_points,fantasy_points_ppr
0,00-0000865,Charlie Batch,QB,QB,2,REG,2010,TEN,PIT,1,0.0,0,1.0,0.0,0.0,-0.982634,0,0,0,0.0,0,1.000000,1.000000
1,00-0000865,Charlie Batch,QB,QB,3,REG,2010,TB,PIT,5,26.0,0,0.0,0.0,2.0,3.388114,0,0,0,0.0,0,18.040001,18.040001
2,00-0000865,Charlie Batch,QB,QB,4,REG,2010,BAL,PIT,1,4.0,0,0.0,0.0,0.0,-0.714745,0,0,0,0.0,0,4.040000,4.040000
3,00-0001231,Josh Bidwell,P,SPEC,1,REG,2010,DAL,WAS,1,0.0,0,1.0,0.0,0.0,-3.118365,0,0,0,0.0,0,0.000000,0.000000
4,00-0001545,Todd Bouman,QB,QB,7,REG,2010,KC,JAX,2,8.0,0,0.0,0.0,1.0,2.377490,0,0,0,0.0,0,13.680000,13.680000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2978,00-0039921,Trey Benson,RB,RB,4,REG,2024,WAS,ARI,9,50.0,0,1.0,0.0,3.0,-1.033874,0,0,0,0.0,0,5.000000,5.000000
2979,00-0039921,Trey Benson,RB,RB,6,REG,2024,GB,ARI,5,26.0,0,0.0,0.0,1.0,-0.513847,0,0,0,0.0,0,2.600000,2.600000
2980,00-0039921,Trey Benson,RB,RB,8,REG,2024,MIA,ARI,1,10.0,0,0.0,0.0,1.0,0.690985,0,0,0,0.0,0,1.000000,1.000000
2981,00-0039921,Trey Benson,RB,RB,9,REG,2024,CHI,ARI,8,37.0,1,0.0,0.0,2.0,0.168990,0,1,1,18.0,0,11.500000,12.500000


In [311]:
player_fantasy[['ppr_roll', 'rush_roll', 'carries_roll']] = player_fantasy.groupby(context_cols)[['fantasy_points_ppr', 'rushing_yards', 'carries']].transform(lambda x: x.rolling(6, min_periods=0).mean())
player_fantasy.loc[player_fantasy.player_display_name=='Charlie Batch']

,player_id,player_display_name,position,position_group,week,season_type,season,opponent_team,recent_team,carries,rushing_yards,rushing_tds,rushing_fumbles,rushing_fumbles_lost,rushing_first_downs,rushing_epa,rushing_2pt_conversions,receptions,targets,receiving_yards,receiving_tds,fantasy_points,fantasy_points_ppr,ppr_roll,rush_roll,carries_roll
0,00-0000865,Charlie Batch,QB,QB,2,REG,2010,TEN,PIT,1,0.0,0,1.0,0.0,0.0,-0.982634,0,0,0,0.0,0,1.000000,1.000000,1.000000,0.000000,1.000000
1,00-0000865,Charlie Batch,QB,QB,3,REG,2010,TB,PIT,5,26.0,0,0.0,0.0,2.0,3.388114,0,0,0,0.0,0,18.040001,18.040001,9.520000,13.000000,3.000000
2,00-0000865,Charlie Batch,QB,QB,4,REG,2010,BAL,PIT,1,4.0,0,0.0,0.0,0.0,-0.714745,0,0,0,0.0,0,4.040000,4.040000,7.693334,10.000000,2.333333
1,00-0000865,Charlie Batch,QB,QB,13,REG,2011,CIN,PIT,3,-2.0,0,0.0,0.0,0.0,0.000000,0,0,0,0.0,0,-0.200000,-0.200000,5.720000,7.000000,2.500000
2,00-0000865,Charlie Batch,QB,QB,14,REG,2011,CLE,PIT,0,0.0,0,0.0,0.0,0.0,NaN,0,0,0,0.0,0,0.000000,0.000000,4.576000,5.600000,2.000000
3,00-0000865,Charlie Batch,QB,QB,16,REG,2011,LA,PIT,0,0.0,0,0.0,0.0,0.0,NaN,0,0,0,0.0,0,6.320000,6.320000,4.866667,4.666667,1.666667
0,00-0000865,Charlie Batch,QB,QB,12,REG,2012,CLE,PIT,0,0.0,0,0.0,0.0,0.0,NaN,0,0,0,0.0,0,1.960000,1.960000,5.026667,4.666667,1.500000
1,00-0000865,Charlie Batch,QB,QB,13,REG,2012,BAL,PIT,0,0.0,0,0.0,0.0,0.0,NaN,0,0,0,0.0,0,13.040000,13.040000,4.193333,0.333333,0.666667


In [312]:
player_fantasy[context_cols + game_context + ['ppr_roll', 'rush_roll']].to_parquet('data/agg/player_weekly_agg.parquet')

In [321]:
player_fantasy.loc[
    (player_fantasy.recent_team=='PHI') &
    (player_fantasy.season==2024) & 
    (player_fantasy.week==11)
]

,player_id,player_display_name,position,position_group,week,season_type,season,opponent_team,recent_team,carries,rushing_yards,rushing_tds,rushing_fumbles,rushing_fumbles_lost,rushing_first_downs,rushing_epa,rushing_2pt_conversions,receptions,targets,receiving_yards,receiving_tds,fantasy_points,fantasy_points_ppr,ppr_roll,rush_roll,carries_roll
668,00-0034351,Dallas Goedert,TE,TE,11,REG,2024,WAS,PHI,0,0.0,0,0.0,0.0,0.0,NaN,0,5,5,61.0,0,6.100000,11.100000,11.433333,0.000000,0.000000
823,00-0034844,Saquon Barkley,RB,RB,11,REG,2024,WAS,PHI,26,146.0,2,0.0,0.0,7.0,6.147048,0,2,3,52.0,0,31.799999,33.799999,20.283334,117.000000,20.666667
1090,00-0035676,A.J. Brown,WR,WR,11,REG,2024,WAS,PHI,0,0.0,0,0.0,0.0,0.0,NaN,0,5,8,65.0,0,6.500000,11.500000,14.983333,0.000000,0.000000
1354,00-0036389,Jalen Hurts,QB,QB,11,REG,2024,WAS,PHI,10,39.0,1,0.0,0.0,4.0,3.295247,0,0,0,0.0,0,18.740000,18.740000,26.346667,42.333333,10.166667
1581,00-0036912,DeVonta Smith,WR,WR,11,REG,2024,WAS,PHI,0,0.0,0,0.0,0.0,0.0,NaN,0,4,6,29.0,0,2.900000,6.900000,10.950000,0.000000,0.000000
1591,00-0036919,Kenneth Gainwell,RB,RB,11,REG,2024,WAS,PHI,4,43.0,0,0.0,0.0,2.0,1.962898,0,1,1,6.0,0,4.900000,5.900000,4.116667,30.000000,6.166667
1923,00-0037741,Jahan Dotson,WR,WR,11,REG,2024,WAS,PHI,0,0.0,0,0.0,0.0,0.0,NaN,0,1,2,8.0,0,0.800000,1.800000,2.750000,2.166667,0.166667


In [322]:
depth = pd.read_csv('data/depth.csv', index_col=0).sort_values(by='depth_team')
depth['dense_depth'] = depth.groupby(['club_code','season','week','position', 'game_type','formation']).cumcount()

In [324]:
off = depth.loc[depth.formation=='Offense']

In [339]:
agg = player_fantasy[['recent_team','season','week','position','player_id','ppr_roll', 'rush_roll', 'carries_roll']].copy()
agg = off.merge(agg,how='left', left_on=['gsis_id', 'club_code', 'season','week','position'],right_on=['player_id', 'recent_team','season','week','position'])
agg = agg.sort_values(by=['club_code','season','week','full_name'])
agg[[ 'player_id']] = agg.groupby('elias_id')[['player_id']].ffill()
agg[['recent_team', 'ppr_roll', 'rush_roll', 'carries_roll','player_id']] = agg.groupby('elias_id')[['recent_team', 'ppr_roll', 'rush_roll', 'carries_roll', 'player_id']].bfill()
agg[['recent_team', 'ppr_roll', 'rush_roll', 'carries_roll','player_id']] = agg.groupby('elias_id')[['recent_team', 'ppr_roll', 'rush_roll', 'carries_roll','player_id']].ffill()
agg[[ 'ppr_roll', 'rush_roll', 'carries_roll']] = agg[[ 'ppr_roll', 'rush_roll', 'carries_roll']].fillna(0)
agg.loc[agg.full_name=='Colson Yankoff']

,season,club_code,week,game_type,depth_team,last_name,first_name,football_name,formation,gsis_id,jersey_number,position,elias_id,depth_position,full_name,dense_depth,recent_team,player_id,ppr_roll,rush_roll,carries_roll
226834,2024,WAS,1.0,REG,3,Yankoff,Colson,Colson,Offense,00-0039686,41.0,RB,YAN546972,\n,Colson Yankoff,3.0,NaN,NaN,0.0,0.0,0.0
226861,2024,WAS,2.0,REG,3,Yankoff,Colson,Colson,Offense,00-0039686,41.0,RB,YAN546972,\n,Colson Yankoff,3.0,NaN,NaN,0.0,0.0,0.0
226895,2024,WAS,3.0,REG,3,Yankoff,Colson,Colson,Offense,00-0039686,41.0,RB,YAN546972,\n,Colson Yankoff,3.0,NaN,NaN,0.0,0.0,0.0
226928,2024,WAS,4.0,REG,3,Yankoff,Colson,Colson,Offense,00-0039686,41.0,RB,YAN546972,\n,Colson Yankoff,3.0,NaN,NaN,0.0,0.0,0.0
226961,2024,WAS,5.0,REG,3,Yankoff,Colson,Colson,Offense,00-0039686,41.0,RB,YAN546972,\n,Colson Yankoff,2.0,NaN,NaN,0.0,0.0,0.0
227008,2024,WAS,6.0,REG,3,Yankoff,Colson,Colson,Offense,00-0039686,41.0,RB,YAN546972,\n,Colson Yankoff,3.0,NaN,NaN,0.0,0.0,0.0
227038,2024,WAS,7.0,REG,3,Yankoff,Colson,Colson,Offense,00-0039686,41.0,RB,YAN546972,\n,Colson Yankoff,2.0,NaN,NaN,0.0,0.0,0.0
227052,2024,WAS,8.0,REG,3,Yankoff,Colson,Colson,Offense,00-0039686,41.0,RB,YAN546972,\n,Colson Yankoff,2.0,NaN,NaN,0.0,0.0,0.0
227106,2024,WAS,9.0,REG,3,Yankoff,Colson,Colson,Offense,00-0039686,41.0,RB,YAN546972,\n,Colson Yankoff,3.0,NaN,NaN,0.0,0.0,0.0
227137,2024,WAS,10.0,REG,3,Yankoff,Colson,Colson,Offense,00-0039686,41.0,RB,YAN546972,\n,Colson Yankoff,3.0,NaN,NaN,0.0,0.0,0.0


In [340]:
tb_agg = agg.loc[agg.club_code=='TB'].copy(deep=True)
tb_agg = tb_agg.loc[tb_agg.position=='RB']
tb_agg.groupby(['club_code','season','week','position', 'gsis_id', 'game_type']).count().sort_values(by='game_type')

depth_team  last_name  \
club_code season week position gsis_id    game_type                          
TB        2023   21.0 RB       00-0034681 CON                 1          1   
                               00-0038951 CON                 1          1   
                               00-0037256 CON                 1          1   
          2021   21.0 RB       00-0036450 CON                 1          1   
                               00-0034816 CON                 1          1   
...                                                         ...        ...   
          2023   19.0 RB       00-0034681 WC                  1          1   
          2021   19.0 RB       00-0030496 WC                  1          1   
                               00-0034816 WC                  1          1   
          2022   19.0 RB       00-0030456 WC                  1          1   
          2021   19.0 RB       00-0030465 WC                  1          1   

                                                     first_name  \
club_code season week position gsis_id    game_type               
TB        2023   21.0 RB       00-0034681 CON                 1   
                               00-0038951 CON                 1   
                               00-0037256 CON                 1   
          2021   21.0 RB       00-0036450 CON                 1   
                               00-0034816 CON                 1   
...                                                         ...   
          2023   19.0 RB       00-0034681 WC                  1   
          2021   19.0 RB       00-0030496 WC                  1   
                               00-0034816 WC                  1   
          2022   19.0 RB       00-0030456 WC                  1   
          2021   19.0 RB       00-0030465 WC                  1   

                                                     football_name  formation  \
club_code season week position gsis_id    game_type                             
TB        2023   21.0 RB       00-0034681 CON                    1          1   
                               00-0038951 CON                    1          1   
                               00-0037256 CON                    1          1   
          2021   21.0 RB       00-0036450 CON                    1          1   
                               00-0034816 CON                    1          1   
...                                                            ...        ...   
          2023   19.0 RB       00-0034681 WC                     1          1   
          2021   19.0 RB       00-0030496 WC                     1          1   
                               00-0034816 WC                     1          1   
          2022   19.0 RB       00-0030456 WC                     1          1   
          2021   19.0 RB       00-0030465 WC                     1          1   

                                                     jersey_number  elias_id  \
club_code season week position gsis_id    game_type                            
TB        2023   21.0 RB       00-0034681 CON                    1         1   
                               00-0038951 CON                    1         1   
                               00-0037256 CON                    1         1   
          2021   21.0 RB       00-0036450 CON                    1         1   
                               00-0034816 CON                    1         1   
...                                                            ...       ...   
          2023   19.0 RB       00-0034681 WC                     1         1   
          2021   19.0 RB       00-0030496 WC                     1         1   
                               00-0034816 WC                     1         1   
          2022   19.0 RB       00-0030456 WC                     1         1   
          2021   19.0 RB       00-0030465 WC                     1         1   

                                                     depth_position  \


In [341]:
tb_agg.loc[
    (tb_agg.season==2024) &
    (tb_agg.week == 9)
]

,season,club_code,week,game_type,depth_team,last_name,first_name,football_name,formation,gsis_id,jersey_number,position,elias_id,depth_position,full_name,dense_depth,recent_team,player_id,ppr_roll,rush_roll,carries_roll
158230,2024,TB,9.0,REG,2,Irving,Mar'Keise,Bucky,Offense,00-0039361,7.0,RB,IRV731762,RB,Bucky Irving,1.0,TB,00-0039361,12.516667,44.166667,9.666667
519,2024,TB,9.0,REG,1,White,Rachaad,Rachaad,Offense,00-0037256,1.0,RB,WHI393110,RB,Rachaad White,0.0,TB,00-0037256,14.300000,37.666667,7.500000
226405,2024,TB,9.0,REG,3,Tucker,Sean,Sean,Offense,00-0038951,44.0,RB,TUC299932,RB,Sean Tucker,2.0,TB,00-0038951,7.883334,31.000000,4.333333


In [342]:
def stack_players(df, num_players, index_cols, value_cols):
    'df should be seperated by position already'
    df = df.loc[df.dense_depth < num_players]
    piv = df.pivot(index=index_cols, columns=['position','dense_depth'], values=value_cols).reset_index()
    piv.columns = [x[0] + '_' + x[1] +'_' +  (str(int(x[2]))) if x[2] != '' else x[0] + x[1] for x in piv.columns ]
    return piv

In [343]:
rb_stats = stack_players(agg.loc[(agg.position=='RB') & (~agg.week.isna())],3,  ['club_code','season','week', 'game_type'], ['gsis_id','full_name', 'ppr_roll','rush_roll', 'carries_roll'])
rb_stats = rb_stats.rename({'club_code':'team_name'},axis=1)
rb_stats

,team_name,season,week,game_type,gsis_id_RB_1,gsis_id_RB_2,gsis_id_RB_0,full_name_RB_1,full_name_RB_2,full_name_RB_0,ppr_roll_RB_1,ppr_roll_RB_2,ppr_roll_RB_0,rush_roll_RB_1,rush_roll_RB_2,rush_roll_RB_0,carries_roll_RB_1,carries_roll_RB_2,carries_roll_RB_0
0,ARI,2010,1.0,REG,00-0027007,00-0026956,00-0026289,Chris Wells,LaRod Stephens-Howling,Tim Hightower,5.5,9.5,15.4,47.0,49.0,54.0,9.5,7.0,13.0
1,ARI,2010,2.0,REG,00-0027007,00-0026956,00-0026289,Chris Wells,LaRod Stephens-Howling,Tim Hightower,5.5,7.5,17.9,47.0,24.5,84.5,9.5,3.5,12.0
2,ARI,2010,3.0,REG,00-0022288,00-0026956,00-0026289,Jason Wright,LaRod Stephens-Howling,Tim Hightower,1.05,7.0,13.6,1.5,16.333333,69.666667,1.0,2.333333,11.666667
3,ARI,2010,4.0,REG,00-0027007,00-0026956,00-0026289,Chris Wells,LaRod Stephens-Howling,Tim Hightower,5.5,5.35,10.8,47.0,13.25,58.25,9.5,2.25,10.5
4,ARI,2010,6.0,REG,00-0027007,00-0022288,00-0026289,Chris Wells,Jason Wright,Tim Hightower,6.85,2.333333,8.183333,45.75,1.0,49.666667,13.25,0.666667,8.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8873,WAS,2024,8.0,REG,00-0033699,00-0039686,00-0037746,Austin Ekeler,Colson Yankoff,Brian Robinson,9.916667,0.0,13.516667,38.333333,0.0,70.166667,6.166667,0.0,14.833333
8874,WAS,2024,9.0,REG,00-0033699,00-0033955,00-0037746,Austin Ekeler,Jeremy McNichols,Brian Robinson,10.883333,6.3,13.45,39.0,30.333333,58.5,6.666667,5.5,14.666667
8875,WAS,2024,10.0,REG,00-0033699,00-0033955,00-0037746,Austin Ekeler,Jeremy McNichols,Brian Robinson,11.616667,4.016667,13.45,40.5,20.166667,58.5,8.333333,4.833333,14.666667
8876,WAS,2024,11.0,REG,00-0033699,00-0033955,00-0037746,Austin Ekeler,Jeremy McNichols,Brian Robinson,12.6,2.55,13.45,30.5,13.666667,58.5,7.666667,4.166667,14.666667


In [344]:
rb_stats.to_csv('data/rb_stats.csv')

In [345]:
rb_stats.dtypes

team_name             object
season                 int64
week                 float64
game_type             object
gsis_id_RB_1          object
gsis_id_RB_2          object
gsis_id_RB_0          object
full_name_RB_1        object
full_name_RB_2        object
full_name_RB_0        object
ppr_roll_RB_1         object
ppr_roll_RB_2         object
ppr_roll_RB_0         object
rush_roll_RB_1        object
rush_roll_RB_2        object
rush_roll_RB_0        object
carries_roll_RB_1     object
carries_roll_RB_2     object
carries_roll_RB_0     object
dtype: object